In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

In [2]:
data = pd.read_excel('Final Dataset.xlsx')

In [3]:
# separate exogenous variables for all companies from the prices volumes and tradig ranges

exogenous = data[['Close_1YBond', 'Close_10YBond', 'Unemployment rate', 'Money supply (M2)', 
    'Reference rate', 'CPI',
    'GDP per capita USD', 'Close_EUR/PLN', 'Close_Gold', 'Close_Oil',
    'Close_USD/PLN', 'Close_VIX']]

stocks = data.loc[:, :'Trading range_sWIG80']

In [4]:
pko = stocks['Close_PKO']
pko_exo = pd.concat((stocks[['Volume_PKO', 'Trading range_PKO']],exogenous), axis=1)

In [39]:
# set plotting style
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 9

data = pd.read_excel('Final Dataset.xlsx')

data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])

exogenous = data[['Close_1YBond', 'Close_10YBond', 'Unemployment rate', 'Money supply (M2)', 
    'Reference rate', 'CPI',
    'GDP per capita USD', 'Close_EUR/PLN', 'Close_Gold', 'Close_Oil',
    'Close_USD/PLN', 'Close_VIX']]

data = data.iloc[:,[0, 1, 2]]
data = pd.concat((data, exogenous), axis=1)
data = data.iloc[1:,:]

# Prophet requires column names to be 'ds' & 'y'
data.rename(columns={'Unnamed: 0': 'ds', 'Close_PKO': 'y'}, inplace=True)

data.sample(5)

,ds,y,Volume_PKO,Close_1YBond,Close_10YBond,Unemployment rate,Money supply (M2),Reference rate,CPI,GDP per capita USD,Close_EUR/PLN,Close_Gold,Close_Oil,Close_USD/PLN,Close_VIX
104,2015-05-27,33.000000,8936718,1663,2965.0,10.766667,1.051191e+06,1.5,-0.010226,12560.051420,4.14410,1185.900024,57.509998,3.80770,13.270000
873,2018-05-29,39.099998,2303796,1351,3297.0,6.120000,1.329617e+06,1.5,0.016933,15504.507506,4.29966,1298.699951,66.730003,3.69826,17.020000
945,2018-09-06,41.700001,2172522,1395,3285.0,5.783333,1.353952e+06,1.5,0.020000,15504.507506,4.32500,1197.900024,67.769997,3.71715,14.650000
370,2016-06-10,23.480000,1496969,1591,3107.0,8.950000,1.184913e+06,1.5,-0.009069,12378.812811,4.33325,1273.400024,49.070000,3.83041,17.030001
1163,2019-07-09,41.799999,1846573,1262,2199.0,5.273913,1.468402e+06,1.5,0.026000,15700.013580,4.25660,1397.500000,57.830002,3.79500,14.090000


In [38]:


# Define additional performance metrics
def mean_directional_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true[1:] - y_true[:-1]) == np.sign(y_pred[1:] - y_pred[:-1]))

def hit_rate(y_true, y_pred, tolerance=0.01):
    y_true = np.squeeze(y_true)
    error = np.abs(y_true - y_pred) / y_true
    hits = np.sum(error <= tolerance)
    return (hits / len(y_true)) * 100

# Define parameters for tuning
grid = ParameterGrid(
    {
        'seasonality_mode': ['additive', 'multiplicative'],
        'n_changepoints': [10, 25, 30],
        'changepoint_range': [0.8, 0.9],
    }
)

# Define the function for extracting regressors from the dataframe
def exogenous_list(df):
    reg_list = df.columns.tolist()
    reg_list.remove('ds')
    reg_list.remove('y')
    return reg_list

# Extract the regressors dynamically
train = data.query('ds < "2019-01-01"')
test = data.query('ds >= "2019-01-01"')
regressors = exogenous_list(train)

# Prepare the lists to store results
results = []

# Loop over the parameter grid
count = 0
for param in grid:
    count += 1
    print(f"Fitting model {count} with parameters: {param}")

    # Initialize the Prophet model with current parameters
    train_model = Prophet(
        weekly_seasonality=True,
        daily_seasonality=False,
        yearly_seasonality=True,
        holidays=None,  # We'll add holidays manually below
        interval_width=0.95,
        changepoint_range=param['changepoint_range'],
        n_changepoints=param['n_changepoints'],
        seasonality_mode=param['seasonality_mode'],
        stan_backend='CMDSTANPY'
    )
    
    # Add Polish holidays
    train_model.add_country_holidays(country_name='PL')
    
    # Add regressors to the model
    for regressor in regressors:
        if regressor in train.columns:
            train_model.add_regressor(regressor, prior_scale=None, standardize='auto')
        else:
            print(f"Warning: Regressor '{regressor}' is missing in training data")
    
    # Fit the model
    print("Fitting the model...")
    train_model.fit(train)

    # Perform cross-validation using a rolling window
    print("Performing cross-validation...")
    df_cv = cross_validation(
        model=train_model,
        initial='1246 days',  
        horizon='1 day',
        period='1 day'
    )

    # Calculate cross-validation performance metrics
    df_pmetrics = performance_metrics(df_cv)
    rmse = df_pmetrics['rmse'].mean()
    print(f"Mean RMSE: {rmse:.4f}")

    # Extract actual and predicted values from cross-validation
    y_true = df_cv['y']
    y_pred = df_cv['yhat']

    # Calculate additional performance metrics
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mda = mean_directional_accuracy(y_true.values, y_pred.values)
    hr = hit_rate(y_true.values, y_pred.values, tolerance=0.025)

    # Store the results
    results.append({
        'parameters': param,
        'rmse': rmse,
        'mse': mse,
        'mae': mae,
        'mape': mape,
        'mda': mda,
        'hit_rate': hr
    })

# Store results in a DataFrame
model_results = pd.DataFrame(results)

# Output the results
print("\nParameter tuning results:")
print(model_results)


Fitting model 1 with parameters: {'changepoint_range': 0.8, 'n_changepoints': 10, 'seasonality_mode': 'additive'}
Fitting the model...


21:59:29 - cmdstanpy - INFO - Chain [1] start processing
21:59:29 - cmdstanpy - INFO - Chain [1] done processing


Performing cross-validation...


  0%|          | 0/151 [00:00<?, ?it/s]

21:59:29 - cmdstanpy - INFO - Chain [1] start processing
21:59:30 - cmdstanpy - INFO - Chain [1] done processing
21:59:30 - cmdstanpy - INFO - Chain [1] start processing
21:59:30 - cmdstanpy - INFO - Chain [1] done processing
21:59:30 - cmdstanpy - INFO - Chain [1] start processing
21:59:31 - cmdstanpy - INFO - Chain [1] done processing
21:59:31 - cmdstanpy - INFO - Chain [1] start processing
21:59:31 - cmdstanpy - INFO - Chain [1] done processing
21:59:32 - cmdstanpy - INFO - Chain [1] start processing
21:59:32 - cmdstanpy - INFO - Chain [1] done processing
21:59:32 - cmdstanpy - INFO - Chain [1] start processing
21:59:33 - cmdstanpy - INFO - Chain [1] done processing
21:59:33 - cmdstanpy - INFO - Chain [1] start processing
21:59:33 - cmdstanpy - INFO - Chain [1] done processing
21:59:34 - cmdstanpy - INFO - Chain [1] start processing
21:59:34 - cmdstanpy - INFO - Chain [1] done processing
21:59:34 - cmdstanpy - INFO - Chain [1] start processing
21:59:35 - cmdstanpy - INFO - Chain [1]

...

22:26:26 - cmdstanpy - INFO - Chain [1] start processing
22:26:27 - cmdstanpy - INFO - Chain [1] done processing


Performing cross-validation...


  0%|          | 0/151 [00:00<?, ?it/s]

22:26:28 - cmdstanpy - INFO - Chain [1] start processing
22:26:29 - cmdstanpy - INFO - Chain [1] done processing
22:26:29 - cmdstanpy - INFO - Chain [1] start processing
22:26:30 - cmdstanpy - INFO - Chain [1] done processing
22:26:30 - cmdstanpy - INFO - Chain [1] start processing
22:26:32 - cmdstanpy - INFO - Chain [1] done processing
22:26:32 - cmdstanpy - INFO - Chain [1] start processing
22:26:34 - cmdstanpy - INFO - Chain [1] done processing
22:26:34 - cmdstanpy - INFO - Chain [1] start processing
22:26:35 - cmdstanpy - INFO - Chain [1] done processing
22:26:35 - cmdstanpy - INFO - Chain [1] start processing
22:26:37 - cmdstanpy - INFO - Chain [1] done processing
22:26:37 - cmdstanpy - INFO - Chain [1] start processing
22:26:37 - cmdstanpy - INFO - Chain [1] done processing
22:26:38 - cmdstanpy - INFO - Chain [1] start processing
22:26:39 - cmdstanpy - INFO - Chain [1] done processing
22:26:39 - cmdstanpy - INFO - Chain [1] start processing
22:26:40 - cmdstanpy - INFO - Chain [1]

Mean RMSE: 0.9695

Parameter tuning results:
                                           parameters      rmse       mse  \
0   {'changepoint_range': 0.8, 'n_changepoints': 1...  1.163194  1.353021   
1   {'changepoint_range': 0.8, 'n_changepoints': 1...  1.131656  1.280645   
2   {'changepoint_range': 0.8, 'n_changepoints': 2...  1.136996  1.292760   
3   {'changepoint_range': 0.8, 'n_changepoints': 2...  0.988442  0.977018   
4   {'changepoint_range': 0.8, 'n_changepoints': 3...  1.122979  1.261082   
5   {'changepoint_range': 0.8, 'n_changepoints': 3...  1.028424  1.057656   
6   {'changepoint_range': 0.9, 'n_changepoints': 1...  1.180790  1.394264   
7   {'changepoint_range': 0.9, 'n_changepoints': 1...  1.048585  1.099531   
8   {'changepoint_range': 0.9, 'n_changepoints': 2...  1.123151  1.261468   
9   {'changepoint_range': 0.9, 'n_changepoints': 2...  0.991552  0.983176   
10  {'changepoint_range': 0.9, 'n_changepoints': 3...  1.117730  1.249321   
11  {'changepoint_range': 0.9, 